# ML4CPS Project-2 | NB-2

In [ ]:
import os
import sys

basepath = os.path.abspath(os.path.join(".."))
if not basepath in sys.path:
    sys.path.append(basepath)

%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.decomposition import PCA
import seaborn as sns
from sklearn.preprocessing import StandardScaler
%matplotlib inline

In [ ]:
from utils.dataset import DatasetLoaderXL
from utils.unsupervised import (
    kmeans_clustering,
    pca_dimensionality_reduction,
    mds_dimensionality_reduction,
    tsne_dimensionality_reduction,
    plot_clusters_2d,
    plot_clusters_3d,
    evaluate_clustering_performance,
    evaluate_dimensionality_reduction_performance,
)

In [ ]:
SEED = 42

## Load dataset

In [ ]:
dsxl = DatasetLoaderXL(dataset_dir="../dataset")
dsxl.load_all_datasets()

In [ ]:
dsxl.list_categories()

In [ ]:
dsxl.get_category_across_all_suburbs("Geography")
df = dsxl.get_values_for_subcategory_across_all_suburbs()

In [ ]:
subset_df = df.iloc[:, [0, 5, 7]]
subset_df

In [ ]:
subset_df_numeric = subset_df[["Area (km^2)", "Distance to GPO (km)"]].apply(
    pd.to_numeric, errors="coerce"
)
subset_df_numeric = subset_df_numeric.fillna(0)

In [ ]:
clusters = kmeans_clustering(
    subset_df_numeric, n_clusters=5, random_state=SEED
)
clusters

In [ ]:
plot_clusters_2d(subset_df_numeric, clusters)